In [33]:
%%javascript

require(["//cdnjs.cloudflare.com/ajax/libs/d3/3.5.6/d3.min.js"],
    function(d3) {
        window.d3 = d3;
        console.log("loaded d3")
    }
);



<IPython.core.display.Javascript object>

In [34]:
%%javascript

function log(mesg, elem) {
    if (elem) {
        elem.append("<p>" + mesg + "</p>")
    } else {
        console.log(mesg)
    }
}

function toPy(value, ident) {
    var kernel = IPython.notebook.kernel;
    value = value.replace("'", "\\'");
    value = value.replace('"', '\\"');
    var code_input = ident + " = '" + value + "'";
    var msg_id = kernel.execute(code_input);
}

function toJs(ident, ident2, cb2) {
    var kernel = IPython.notebook.kernel;
    function callback(out){
        console.log(out);
        var res = null;
        
        // if output is a print statement
        if(out.msg_type == "stream"){
            res = out.content.text;
        }
        
        // if output is a python object
        else if(out.msg_type === "pyout"){
            res = out.content.data["text/plain"];
        }
        
        // if output is executing result
        else if(out.msg_type === "execute_result"){
            res = out.content.data["text/plain"];
        }
        
        // if output is a python error
        else if(out.msg_type == "pyerr"){
            res = out.content.ename + ": " + out.content.evalue;
        }
        
        // if output is something we haven't thought of
        else{
            res = "[out type not implemented]";  
        }
        
        if (!ident2) {
            ident2 = ident;
        }
        
        window.nbvars[ident2] = res;
        
        if (cb2) { 
            cb2(res);
        }
    }
    
    var callbacks = { 'iopub' : {'output' : callback}};
    var code_input = ident;
    var msg_id = kernel.execute(code_input, callbacks, {silent:false});
}


window.nbvars = {
    "log": log,
}

window.toPy = toPy
window.toJs = toJs

<IPython.core.display.Javascript object>

In [42]:
%%javascript

function d3_xy_chart() {
    var width = 640,  
        height = 480, 
        xlabel = "X Axis Label",
        ylabel = "Y Axis Label" ;
    
    function chart(selection) {
        selection.each(function(datasets) {
            //
            // Create the plot. 
            //
            var margin = {top: 20, right: 80, bottom: 30, left: 50}, 
                innerwidth = width - margin.left - margin.right,
                innerheight = height - margin.top - margin.bottom ;
            
            var x_scale = d3.scale.linear()
                .range([0, innerwidth])
                .domain([ d3.min(datasets, function(d) { return d3.min(d.x); }), 
                          d3.max(datasets, function(d) { return d3.max(d.x); }) ]) ;
            
            var y_scale = d3.scale.linear()
                .range([innerheight, 0])
                .domain([ d3.min(datasets, function(d) { return d3.min(d.y); }),
                          d3.max(datasets, function(d) { return d3.max(d.y); }) ]) ;

            var color_scale = d3.scale.category10()
                .domain(d3.range(datasets.length)) ;

            var x_axis = d3.svg.axis()
                .scale(x_scale)
                .orient("bottom") ;

            var y_axis = d3.svg.axis()
                .scale(y_scale)
                .orient("left") ;

            var x_grid = d3.svg.axis()
                .scale(x_scale)
                .orient("bottom")
                .tickSize(-innerheight)
                .tickFormat("") ;

            var y_grid = d3.svg.axis()
                .scale(y_scale)
                .orient("left") 
                .tickSize(-innerwidth)
                .tickFormat("") ;

            var draw_line = d3.svg.line()
                .interpolate("basis")
                .x(function(d) { return x_scale(d[0]); })
                .y(function(d) { return y_scale(d[1]); }) ;

            var svg = d3.select(this)
                .attr("width", width)
                .attr("height", height)
                .append("g")
                .attr("transform", "translate(" + margin.left + "," + margin.top + ")") ;
            
            svg.append("g")
                .attr("class", "x grid")
                .attr("transform", "translate(0," + innerheight + ")")
                .call(x_grid) ;

            svg.append("g")
                .attr("class", "y grid")
                .call(y_grid) ;

            svg.append("g")
                .attr("class", "x axis")
                .attr("transform", "translate(0," + innerheight + ")") 
                .call(x_axis)
                .append("text")
                .attr("dy", "-.71em")
                .attr("x", innerwidth)
                .style("text-anchor", "end")
                .text(xlabel) ;
            
            svg.append("g")
                .attr("class", "y axis")
                .call(y_axis)
                .append("text")
                .attr("transform", "rotate(-90)")
                .attr("y", 6)
                .attr("dy", "0.71em")
                .style("text-anchor", "end")
                .text(ylabel) ;

            var data_lines = svg.selectAll(".d3_xy_chart_line")
                .data(datasets.map(function(d) {return d3.zip(d.x, d.y);}))
                .enter().append("g")
                .attr("class", ".d3_xy_chart_line") ;
            
            data_lines.append("path")
                .attr("class", "line")
                .attr("d", function(d) {return draw_line(d); })
                .attr("stroke", function(_, i) {return color_scale(i);}) ;
            
            data_lines.append("text")
                .datum(function(d, i) { return {name: datasets[i].label, final: d[d.length-1]}; }) 
                .attr("transform", function(d) { 
                    return ( "translate(" + x_scale(d.final[0]) + "," + 
                             y_scale(d.final[1]) + ")" ) ; })
                .attr("x", 3)
                .attr("dy", ".35em")
                .attr("fill", function(_, i) { return color_scale(i); })
                .text(function(d) { return d.name; }) ;

        }) ;
    }

    chart.width = function(value) {
        if (!arguments.length) return width;
        width = value;
        return chart;
    };

    chart.height = function(value) {
        if (!arguments.length) return height;
        height = value;
        return chart;
    };

    chart.xlabel = function(value) {
        if(!arguments.length) return xlabel ;
        xlabel = value ;
        return chart ;
    } ;

    chart.ylabel = function(value) {
        if(!arguments.length) return ylabel ;
        ylabel = value ;
        return chart ;
    } ;

    return chart;
}

d3.my_d3_xy_chart = d3_xy_chart;


<IPython.core.display.Javascript object>

In [35]:
from IPython.display import HTML

style = """
<style>

body {
  font: 12px sans-serif;
}

.axis path,
.axis line {
  fill: none;
  stroke: #000;
  shape-rendering: crispEdges;
}

.grid path,
.grid line {
  fill: none;
  stroke: rgba(0, 0, 0, 0.25);
  shape-rendering: crispEdges;
}

.x.axis path {
  display: none;
}

.line {
  fill: none;
  stroke-width: 2.5px;
}

</style>

"""

HTML(style)


In [36]:
yval1 = range(5)
yval2 = [x*x*x for x in range(5)]

In [37]:
%%javascript
toJs("yval1")
toJs("yval2")


<IPython.core.display.Javascript object>

In [43]:
%%javascript

nbvars.yval1 = eval(nbvars.yval1)
nbvars.yval2 = eval(nbvars.yval2)

var data = [ { label: "Data Set 1", 
               x: [0, 1, 2, 3, 4], 
               y: nbvars.yval1 }, 
             { label: "Data Set 2", 
               x: [0, 1, 2, 3, 4], 
               y: nbvars.yval2 } ];

var xy_chart = d3.my_d3_xy_chart()
    .width(600)
    .height(200)
    .xlabel("X Axis")
    .ylabel("Y Axis");

$('#d3-chart').empty();

var svg = d3.select("#d3-chart").append("svg")
    .datum(data)
    .call(xy_chart);


<IPython.core.display.Javascript object>

In [22]:
%%javascript
element.append("<div id=d3-chart></div>")

<IPython.core.display.Javascript object>